 - ### 데이터 변환



- 데이터 원본 사이트
http://openapi.nsdi.go.kr/nsdi/eios/OpenapiList.do?provOrg=NIA&gubun=F
    - gis 건물통합정보
    - 연속지적도형정보
    
<hr>

- 데이터 원본 geojson 으로 변환
https://mapshaper.org/
    - encoding=euc-kr


In [1]:
import json

# 002 - 필지, 010 - 건물

with open("AL_11_D010_20220702.json", encoding="utf-8") as f:
    map_data_all = json.load(f)

def flatten_list(list_instance):
    if list_instance == []:
        return list_instance
    if isinstance(list_instance[0], list):
        return flatten_list(list_instance[0]) + flatten_list(list_instance[1:])
    return list_instance[:1] + flatten_list(list_instance[1:])

In [2]:
map_data_filtered = {"type": map_data_all["type"], "features": []}

for each_building in map_data_all["features"]:

    coordinates = each_building["geometry"]["coordinates"]
    flattened_coordinates = flatten_list(coordinates)
    lon_s, lat_s = flattened_coordinates[0], flattened_coordinates[1]

    # if (not 125 < lon_s < 130) or (not 34 < lat_s < 39):
    #     # 기준 잡을 때 위도 경도 맞는지 체크 - 0개
    #     print("위도 경도 맞지 않음")
    #     continue
    if each_building["geometry"]["type"] != "Polygon":
        # MultiPolygon 4개 존재. 편의상 극소수임으로 무시
        print("폴리곤 아님")
        continue

    map_data_filtered["features"].append(each_building)

폴리곤 아님
폴리곤 아님
폴리곤 아님
폴리곤 아님


In [3]:
y_min, x_min, y_max, x_max = 100000000000000, 100000000000000, -100000000000000, -100000000000000

for each_building in map_data_filtered["features"]:

    coordinates = each_building["geometry"]["coordinates"]
    flattened_coordinates = flatten_list(coordinates)
    x_s, y_s = flattened_coordinates[0], flattened_coordinates[1]

    # 이상한 데이터 섞여있음
    if x_s > y_s:
        continue

    if y_min > y_s:
        y_min = y_s
    
    if x_min > x_s:
        x_min = x_s
    
    if y_max < y_s:
        y_max = y_s

    if x_max < x_s:
        x_max = x_s

# 10 * 10으로 나누기 위해 최대 최소 확인
x_min, y_min, x_max, y_max

(179404.39400000032, 436476.5030000005, 219589.24000000022, 466307.8200000003)

In [4]:
division_num = 50

x_interval = (x_max - x_min) / division_num
y_interval = (y_max - y_min) / division_num

x_list = [x_min + x_interval * i for i in range(division_num + 1)]
y_list = [y_min + y_interval * i for i in range(division_num + 1)]

divided_map_data = {}
for building_index, each_building in enumerate(map_data_filtered["features"]):
    coordinates = each_building["geometry"]["coordinates"]
    flattened_coordinates = flatten_list(coordinates)
    x_s, y_s = flattened_coordinates[0], flattened_coordinates[1]

    breaker = False

    for i in range(division_num):
        if y_list[i] <= y_s < y_list[i + 1]:
            for j in range(division_num):
                if x_list[j] <= x_s < x_list[j + 1]:

                    if (x_list[j], y_list[i], x_list[j + 1], y_list[i + 1]) in divided_map_data:
                        divided_map_data[(x_list[j], y_list[i], x_list[j + 1], y_list[i + 1])]["features"].append(each_building)
                        breaker = True
                        break
                    else:
                        divided_map_data[(x_list[j], y_list[i], x_list[j + 1], y_list[i + 1])] = {"type": map_data_all["type"], "features": [each_building]}
                        breaker = True
                        break
        
        if breaker:
            break


In [13]:
[len(divided_map_data[key]["features"]) for key in divided_map_data]

[15200,
 6590,
 35865,
 22736,
 95,
 25894,
 17715,
 10121,
 8901,
 13412,
 14535,
 26931,
 21641,
 11291,
 7465,
 11844,
 8373,
 26841,
 8009,
 9135,
 1557,
 26016,
 13261,
 794,
 78,
 10144,
 4788,
 57,
 131,
 3580,
 192,
 20948,
 7735,
 19447,
 602,
 20687,
 8326,
 12840,
 18145,
 15905,
 11032,
 11833,
 3068,
 1958,
 3155,
 4,
 1,
 9843,
 800,
 22972,
 1999,
 3707,
 28743,
 14179,
 154,
 12337,
 3909,
 9461,
 5772,
 24,
 1135,
 2222,
 16,
 12498,
 6804,
 170,
 3789,
 9636,
 289,
 102,
 40]

In [5]:
for loc_key in divided_map_data:
    with open(f"building_data_divided/{loc_key[0]}_{loc_key[1]}_{loc_key[2]}_{loc_key[3]}.json", "w", encoding="utf-8") as f:
        json.dump(divided_map_data[loc_key], f)